# Currently ran the script for a few hours and it stopped for no reason. However, I do not think there is an issue in the code. I will run it again tommorow and upload the saved data as .csv. I'm scraping the url for every car brand, then for every model of every brand and then for every variant of every model of every brand. I then collect information I considered useful for every variant in a dataframe.


In [7]:
!pip install beautifulsoup4


In [8]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [9]:
# Brand URLs

def fetch_brand_urls():
    url = "https://www.cars-data.com/en/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    brand_divs = soup.find_all("div", class_="col-2")
    brands = []

    for div in brand_divs:
        a_tag = div.find("a", href=True, title=True)
        if a_tag:
            brand_name = a_tag["title"].strip()
            brand_url = a_tag["href"]

            if brand_name.lower() == "station wagon cars":
                continue

            brands.append((brand_name, brand_url))

    return brands

brand_links = fetch_brand_urls()

def generate_brand_urls(brands):
    return [url for _, url in brands]

def print_brands_and_urls(brands):
    for name, url in brands:
      print(f"{name}: {url}")

# print_brands_and_urls(brand_links)

Abarth: https://www.cars-data.com/en/abarth
Aiways: https://www.cars-data.com/en/aiways
Alfa Romeo: https://www.cars-data.com/en/alfa-romeo
Alpine: https://www.cars-data.com/en/alpine
Asia Motors: https://www.cars-data.com/en/asia-motors
Aston Martin: https://www.cars-data.com/en/aston-martin
Audi: https://www.cars-data.com/en/audi
Austin: https://www.cars-data.com/en/austin
Autobianchi: https://www.cars-data.com/en/autobianchi
Bentley: https://www.cars-data.com/en/bentley
BMW: https://www.cars-data.com/en/bmw
Bugatti: https://www.cars-data.com/en/bugatti
Buick: https://www.cars-data.com/en/buick
Cadillac: https://www.cars-data.com/en/cadillac
Carver: https://www.cars-data.com/en/carver
Chevrolet: https://www.cars-data.com/en/chevrolet
Chrysler: https://www.cars-data.com/en/chrysler
Citroen: https://www.cars-data.com/en/citroen
Corvette: https://www.cars-data.com/en/corvette
Cupra: https://www.cars-data.com/en/cupra
Dacia: https://www.cars-data.com/en/dacia
Daewoo: https://www.cars-dat

In [10]:
# Model URLs

urls = generate_brand_urls(brand_links)

# Function to get all model links for a brand
def get_model_links(brand_url):
    try:
        # Add a delay to be respectful to the website
        time.sleep(2)

        # Get the page content
        response = requests.get(brand_url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Look for links with a title attribute that might contain model names
        model_links = soup.find_all('a', title=True, href=True)

        brand_name = brand_url.split('/')[-1]
        models = []

        for link in model_links:

            href = link['href']
            model_split = href.split('/')

            if brand_name in href and len(model_split) > 4:

                if '.html' in href or model_split[-1] == brand_url.split('/')[-1]:
                  continue

                models.append({
                    'brand': brand_name,
                    'model': model_split[-1],
                    'title': brand_name + " " + model_split[-1].replace('-',' '),
                    'url': href
                })

        return models

    except Exception as e:
        print(f"Error processing {brand_url}: {str(e)}")
        return []


def generate_model_urls():

  all_models = []

  # Iterate through each brand URL
  for brand_url in urls:
    #  print(f"Processing {brand_url}...")

      model_links = get_model_links(brand_url)
      for links in model_links:
          print(links)
      all_models.extend(model_links)

     # print(f"Found {len(model_links)} models for {brand_url}")

  return all_models

model_links = generate_model_urls()
#print(model_links[:100])

Processing https://www.cars-data.com/en/abarth...
{'brand': 'abarth', 'model': '124-spider', 'title': 'abarth 124 spider', 'url': 'https://www.cars-data.com/en/abarth/124-spider'}
{'brand': 'abarth', 'model': '595', 'title': 'abarth 595', 'url': 'https://www.cars-data.com/en/abarth/595'}
{'brand': 'abarth', 'model': '595-cabrio', 'title': 'abarth 595 cabrio', 'url': 'https://www.cars-data.com/en/abarth/595-cabrio'}
{'brand': 'abarth', 'model': '695', 'title': 'abarth 695', 'url': 'https://www.cars-data.com/en/abarth/695'}
{'brand': 'abarth', 'model': '500c', 'title': 'abarth 500c', 'url': 'https://www.cars-data.com/en/abarth/500c'}
{'brand': 'abarth', 'model': 'punto-evo', 'title': 'abarth punto evo', 'url': 'https://www.cars-data.com/en/abarth/punto-evo'}
{'brand': 'abarth', 'model': '500', 'title': 'abarth 500', 'url': 'https://www.cars-data.com/en/abarth/500'}
{'brand': 'abarth', 'model': 'grande-punto', 'title': 'abarth grande punto', 'url': 'https://www.cars-data.com/en/abarth/gra

In [11]:
# Variant URLs

def extract_variant_links(model_url, model_title):
    try:
        # Add a delay to be respectful to the website
        time.sleep(1)

        # Get the page content
        response = requests.get(model_url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the models section
        models_section = soup.find('section', class_='models')

        if not models_section:
            print(f"Could not find models section for {model_url}")
            return []

        # Find all links with title containing the model title
        variant_links = []
        for link in models_section.find_all('a', href=True, title=True):
            if model_title.lower() in link['title'].lower():
                variant_links.append({
                    'brand': model_title.split()[0],
                    'model': model_title.split()[1] if len(model_title.split()) > 1 else '',
                    'variant_url': link['href'],
                    'variant_title': link['title'],
                    'variant_id': link['href'].split('/')[-1] if '/' in link['href'] else ''
                })

        return variant_links

    except Exception as e:
        print(f"Error processing {model_url}: {str(e)}")
        return []


def process_models_for_variants(models_list):
    all_variants = []

    for model in models_list:
       # print(f"Processing variants for {model['title']}...")
        variants = extract_variant_links(model['url'], model['title'])
        all_variants.extend(variants)
       # print(f"Found {len(variants)} variants for {model['title']}")

    return all_variants

variant_links = process_models_for_variants(model_links)

variant_links

Processing variants for abarth 124 spider...
Found 1 variants for abarth 124 spider
Processing variants for abarth 595...
Found 1 variants for abarth 595
Processing variants for abarth 595 cabrio...
Found 1 variants for abarth 595 cabrio
Processing variants for abarth 695...
Found 2 variants for abarth 695
Processing variants for abarth 500c...
Found 1 variants for abarth 500c
Processing variants for abarth punto evo...
Found 1 variants for abarth punto evo
Processing variants for abarth 500...
Found 1 variants for abarth 500
Processing variants for abarth grande punto...
Found 1 variants for abarth grande punto
Processing variants for aiways u5...
Found 1 variants for aiways u5
Processing variants for alfa-romeo stelvio...
Found 0 variants for alfa-romeo stelvio
Processing variants for alfa-romeo giulia...
Found 0 variants for alfa-romeo giulia
Processing variants for alfa-romeo 4c spider...
Found 0 variants for alfa-romeo 4c spider
Processing variants for alfa-romeo 4c...
Found 0 var

KeyboardInterrupt: 

In [ ]:
# Spec URLs

def extract_specs_links(variant_url, variant_title):
    try:
        # Add a delay to be respectful to the website
        time.sleep(1)

        # Get the page content
        response = requests.get(variant_url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all rows that might contain spec links
        specs_links = []

        # Look for links in h3 tags inside col-6 divs
        for div in soup.find_all('div', class_='col-6'):
            h3 = div.find('h3')
            if h3:
                link = h3.find('a', href=True, title=True)
                if link and variant_title.lower() in link['title'].lower():
                    # Extract year from the title using regex
                    # Looking for a 4-digit year at the beginning of the title
                    year_match = re.search(r'^(\d{4})', link['title'])
                    year = year_match.group(1) if year_match else "Unknown"

                    specs_links.append({
                        'brand': variant_title.split()[0],
                        'year': year,
                        'spec_title': link['title'],
                        'spec_url': link['href'] +'/tech',
                        'fuel_type': div.find_next_sibling('div', class_='col-2').text.strip() if div.find_next_sibling('div', class_='col-2') else '',
                        'transmission': div.find_next_sibling('div', class_='col-2').find_next_sibling('div', class_='col-2').text.strip() if div.find_next_sibling('div', class_='col-2') and div.find_next_sibling('div', class_='col-2').find_next_sibling('div', class_='col-2') else ''
                    })

        return specs_links

    except Exception as e:
        print(f"Error processing {variant_url}: {str(e)}")
        return []

def process_variants_for_specs(variants_list):

    all_specs = []

    for variant in variants_list:
        print(f"Processing specs for {variant['variant_title']}...")
        specs = extract_specs_links(variant['variant_url'], variant['variant_title'])
        all_specs.extend(specs)
        print(f"Found {len(specs)} specifications for {variant['variant_title']}")

    return all_specs

spec_links = process_variants_for_specs(variant_links)

spec_links


In [ ]:
# Spec collection

def process_spec_page(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch {url}")
        return {}

    soup = BeautifulSoup(response.text, 'html.parser')

    # Collect data from tables
    tables = soup.find_all('table')

    specs = {}

    for table in tables:
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if len(cols) == 2:
                raw_key = cols[0].get_text().strip().lower()
                value = cols[1].get_text().strip()

                if value == '' or value is None:
                    continue

                # print(raw_key + " " + value + "\n")

                # GENERAL
                if 'price' in raw_key and 'road' not in raw_key:
                    specs['price'] = value
                elif 'body type' in raw_key:
                    specs['body_type'] = value
                elif 'number of seats' in raw_key:
                    specs['number_of_seats'] = value

                # ENGINE AND FUEL
                elif 'engine/motor type' in raw_key:
                    specs['engine_type'] = value
                elif 'fuel tank capacity' in raw_key:
                    specs['fuel_tank_capacity'] = value
                elif 'fuel type' in raw_key:
                    specs['fuel_type_detail'] = value
                elif 'max torque' in raw_key:
                    specs['max_torque'] = value

                # ELECTRIC
                elif 'battery range' in raw_key:
                    specs['battery_range'] = value
                elif 'battery capacity' in raw_key:
                    specs['battery_capacity'] = value
                elif 'charging time' in raw_key:
                    specs['charging_time'] = value
                elif 'power consumption' in raw_key:
                    specs['power_consumption'] = value

                # POWER
                elif 'max power' in raw_key:
                    specs['max_power'] = value
                elif 'total max. power' in raw_key and 'kw' in raw_key:
                    specs['total_max_power_kw'] = value
                elif 'total max. power' in raw_key and 'hp' in raw_key:
                    specs['total_max_power_hp'] = value

                # PERFORMANCE
                elif 'turbo' in raw_key:
                    specs['turbo'] = value
                elif 'top speed' in raw_key:
                    specs['top_speed'] = value
                elif 'acceleration 0-100' in raw_key:
                    specs['acceleration_0_100_kmh'] = value

                # CONSUMPTION
                elif 'extra-urban consumption' in raw_key:
                    specs['extra_urban_consumption'] = value
                elif 'urban consumption' in raw_key:
                    specs['urban_consumption'] = value
                elif 'combined consumption' in raw_key:
                    specs['combined_consumption'] = value
                elif 'co2 emissions' in raw_key:
                    specs['co2_emissions'] = value

    return specs

def enrich_car_data(car_entries):
    enriched_data = []
    for entry in car_entries:
        url = entry.get('spec_url')
       # print(f"Processing: {entry.get('spec_title')}")

        details = process_spec_page(url)

        # Merge original with extracted details
        enriched_entry = {
            'year': entry.get('year'),
            'spec_title': entry.get('spec_title'),
            # 'spec_url': entry.get('spec_url'),
            'fuel_type': entry.get('fuel_type'),
            'transmission': entry.get('transmission')
        }

        enriched_entry.update({
            'price': details.get('price'),
            'body_type': details.get('body_type'),
            'number_of_seats': details.get('number_of_seats'),
            'engine_type': details.get('engine_type'),
            'fuel_type_detail': details.get('fuel_type_detail'),
            'max_power': details.get('max_power'),
            'total_max_power_kw': details.get('total_max_power_kw'),
            'total_max_power_hp': details.get('total_max_power_hp'),
            'max_torque': details.get('max_torque'),
            'fuel_tank_capacity': details.get('fuel_tank_capacity'),
            'turbo': details.get('turbo'),
            'top_speed': details.get('top_speed'),
            'acceleration_0_100_kmh': details.get('acceleration_0_100_kmh'),
            'urban_consumption': details.get('urban_consumption'),
            'extra_urban_consumption': details.get('extra_urban_consumption'),
            'combined_consumption': details.get('combined_consumption'),
            'co2_emissions': details.get('co2_emissions'),
            'battery_range': details.get('battery_range'),
            'battery_capacity': details.get('battery_capacity'),
            'charging_time': details.get('charging_time'),
            'power_consumption': details.get('power_consumption')
        })

        enriched_data.append(enriched_entry)

    return enriched_data


car_data = enrich_car_data(spec_links)

car_data

In [1]:
# DataFrame Conversion and Saving data

df = pd.DataFrame(car_data)
df.head()
df.to_csv('cars-data-table.csv', index=True)


NameError: name 'pd' is not defined